# Chapter 6: Influence Maximization over Networks

Implement a greedy algorithm on the linear threshold model to solve the influence maximization problem.

In [ ]:
import torch
from torch_geometric.data import Data
from typing import List, Tuple
from torch_geometric.utils import degree

def simulate_spread(data: Data, seed_set: set) -> int:
    """
    Spread simulation in the Linear Threshold Model.
    Args:
        data (Data): The PyG graph data object.
        seed_set (set): The set of seed nodes.
    Returns:
        The total number of activated nodes.
    """
    # TODO: Implement the spread simulation in the Linear Threshold Model.
    num_nodes = data.num_nodes
    device = data.edge_index.device
    
    # 1. 初始化激活状态 (Active Mask)
    active_mask = torch.zeros(num_nodes, dtype=torch.bool, device=device)
    if seed_set:
        seed_indices = torch.tensor(list(seed_set), device=device)
        active_mask[seed_indices] = True
    
    # 2. 获取固定的参数
    # data.edge_index: [2, num_edges], row 0 is src, row 1 is dst
    src, dst = data.edge_index
    
    # data.edge_attr: [num_edges], 对应每条边的权重 b_{uv}
    edge_weights = data.edge_attr
    
    # data.thresholds: [num_nodes], 每个节点的激活阈值
    thresholds = data.thresholds
    
    # 3. 迭代传播
    while True:
        prev_count = active_mask.sum().item()
        
        # --- 计算当前所有节点的受影响力 ---
        
        # A. 找出源节点是活跃的边
        # active_mask[src] 会得到一个形状为 [num_edges] 的布尔向量
        active_src_mask = active_mask[src]
        
        # B. 获取有效权重
        # 只有当源节点活跃时，边的权重才算数；否则为 0
        current_active_weights = edge_weights * active_src_mask.float()
        
        # C. 聚合影响力 (Sum Pooling)
        # 将所有指向 dst 的有效权重加起来
        current_influence = torch.zeros(num_nodes, device=device)
        current_influence.index_add_(0, dst, current_active_weights)
        
        # --- 判定激活 ---
        
        # D. 如果总影响力 >= 阈值，则激活
        newly_active = current_influence >= thresholds
        
        # E. 更新状态 (单调递增：一旦激活，永远激活)
        active_mask = active_mask | newly_active
        
        # --- 收敛检查 ---
        curr_count = active_mask.sum().item()
        if curr_count == prev_count:
            break
            
    return curr_count        

def greedy_influence_maximization(data: Data, k: int) -> Tuple[set, int]:
    """
    Greedy influence maximization for the Linear Threshold Model using PyTorch.
    Args:
        data (Data): The PyG graph data object.
        k (int): Number of seed nodes to select.
    Returns:
        The selected seed nodes and the total spread.
    """
    # TODO: Implement the greedy influence maximization algorithm.
    seed_set = set()
    num_nodes = data.num_nodes
    current_spread = 0
    
    # 贪心循环 k 轮
    for _ in range(k):
        best_node = -1
        max_marginal_gain = -1
        
        # 遍历所有非种子节点作为候选
        candidates = [node for node in range(num_nodes) if node not in seed_set]
        
        for node in candidates:
            # 尝试加入该节点
            temp_seeds = seed_set | {node}
            
            # 因为是确定性的，不需要多次模拟取平均，跑一次即可
            spread = simulate_spread(data, temp_seeds)
            
            # 记录能带来最大总传播量的节点
            if spread > max_marginal_gain:
                max_marginal_gain = spread
                best_node = node
        
        # 将本轮最佳节点加入种子集
        if best_node != -1:
            seed_set.add(best_node)
            current_spread = max_marginal_gain
            
    return seed_set, current_spread

/home/yingziyu/.conda/envs/pyg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Test the instance on page 16 of the slides.
num_nodes = 6
edge_set = [(0, 1), (0, 2), (0, 4), (1, 0), (2, 1), (3, 1), (3, 2), (3, 5), (4, 2), (4, 3), (4, 5), (5, 3), (5, 4)]
edge_weights = torch.tensor([0.6, 0.3, 0.4, 0.6, 0.2, 0.2, 0.1, 0.2, 0.5, 0.3, 0.5, 0.2, 0.5], dtype=torch.float32)
thresholds = torch.tensor([0.1, 0.6, 0.55, 0.4, 0.3, 0.5], dtype=torch.float32)
edges = torch.tensor(list(zip(*edge_set)), dtype=torch.long)
data = Data(edge_index=edges, edge_attr=edge_weights, num_nodes=num_nodes, thresholds=thresholds)
simulate_spread(data, {5})

4

In [3]:
# Test the greedy influence maximization
num_nodes = 10
edge_set = [
    (0, 1), (0, 2), (0, 3), (0, 7),
    (1, 0), (1, 2), (1, 4), (1, 5),
    (2, 0), (2, 5), (2, 6),
    (3, 0), (3, 7), (3, 8),
    (4, 1), (4, 5), (4, 9),
    (5, 1), (5, 4), (5, 6), (5, 9),
    (6, 2), (6, 5), (6, 8),
    (7, 0), (7, 3), (7, 8), (7, 9),
    (8, 3), (8, 6), (8, 7), (8, 9),
    (9, 4), (9, 5), (9, 7), (9, 8)
]
edge_weights = torch.tensor([  # Here we may break the contraints of the LTM
    0.5, 0.3, 0.4, 0.6,
    0.5, 0.2, 0.4, 0.3,
    0.3, 0.2, 0.6,
    0.4, 0.7, 0.5,
    0.4, 0.3, 0.6,
    0.3, 0.5, 0.4, 0.2,
    0.6, 0.3, 0.5,
    0.6, 0.3, 0.7, 0.5,
    0.5, 0.4, 0.7, 0.3,
    0.6, 0.5, 0.4, 0.5
], dtype=torch.float32)

thresholds = torch.tensor([0.6, 0.4, 0.5, 0.3, 0.6, 0.5, 0.4, 0.3, 0.6, 0.5], dtype=torch.float32) + 0.6  # Here we may break the contraints of the LTM
edges = torch.tensor(list(zip(*edge_set)), dtype=torch.long)

data = Data(edge_index=edges, edge_attr=edge_weights, num_nodes=num_nodes, thresholds=thresholds)

for k in range(1, 5):
    seed_set, spread = greedy_influence_maximization(data, k)
    print(seed_set, spread)

{0} 1
{0, 9} 5
{0, 9, 2} 7
{0, 9, 2, 1} 10


---
# Discussions

...